<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/xgboost_rf_live_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,622 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,722 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 59.0 MB/s eta 0:00:00
  Attempting uninstall: cudf-cu12
    Found existing installation: cudf-cu12 25.2.1
    Uninstalling cudf-cu12-25.2.1:
      Successfully uninstalled cudf-cu12-25.2.1
  Attempting uninstall: dask-cudf-cu12
    Found existing installation: dask-cudf-cu12 25.2.2
    Uninstalling dask-cudf-cu12-25.2.2:
      Successfully uninstalled dask-cudf-cu12-25.2.2
  Attempting uninstall: cuml-cu12
    Found existing installation: cuml-cu12 25.2.1
    Uninstalling cuml-cu12-25.2.1:
      Successfully uninstalled cuml-cu12-25.2.1


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [9]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [10]:
#Import Required Libraries
import gc
import json
import os
import random
import time
from collections import deque
from datetime import datetime

import cupy as cp
import cudf
import cuml
import dask
import gymnasium as gym
import gym_anytrading
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
import yfinance as yf
from cuml.ensemble import RandomForestClassifier
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

#Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "sac_ppo_td3_multi_stock_v1"

CONFIG = {
    'symbols': [],
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}
#Import Required Libraries
import gc
import json
import os
import random
import time
from collections import deque
from datetime import datetime

import cupy as cp
import cudf
import cuml
import dask
import gymnasium as gym
import gym_anytrading
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
import yfinance as yf
from cuml.ensemble import RandomForestClassifier
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

#Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]


def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

#Feature Engineering Function
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())
    df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
    df['Rolling_Mean_50'] = df['Close'].rolling(50).mean()
    df['Expanding_Mean'] = df['Close'].expanding().mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['True_Range'] = df[['High', 'Low', 'Close']].apply(
        lambda x: max(x.iloc[0] - x.iloc[1], abs(x.iloc[0] - x.iloc[2]), abs(x.iloc[1] - x.iloc[2])), axis=1)
    df['ATR'] = df['True_Range'].rolling(14).mean()
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
    df['+DI'] = 100 * df['+DM'].rolling(14).mean() / df['ATR']
    df['-DI'] = 100 * df['-DM'].rolling(14).mean() / df['ATR']
    df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(14).mean()
    df['Volume_Avg'] = df['Volume'].rolling(20).mean()
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_Change_MA'] = df['Volume_Change'].rolling(10).mean()
    df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)
    df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']
    df['Trailing_Stop'] = np.minimum(df['Close'] * 0.985, df['Close'] - (df['ATR'] * 0.3))
    df['Buy_Signal'] = np.where((df['RSI'] < 60) & (df['EMA_10'] > df['EMA_50']) &
                                ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
                                (df['Volume'] > (0.4 * df['Volume_Avg'])) & (df['ADX'] > 18), 1, 0)
    df['Sell_Signal'] = np.where(((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
                                 ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
                                 (df['Close'] < df['Trailing_Stop']) |
                                 ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)), 1, 0)
    df['Sell_Signal_Debug'] = np.where(((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
                                       (df['RSI'] < 55) & (df['ADX'] > 15) &
                                       ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
                                       (df['Volume'] > 0.5 * df['Volume_Avg']), 1, 0)
    df['Future_Close'] = df['Close'].shift(-10)
    df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()
    df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df['Multi_Class_Target'] = df['Target']
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
    df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek
    df['Session'] = np.where((df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
                             np.where((df['Hour'] < 9), 'Pre-market', 'After-hours'))
    df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

all_dfs = []

for ticker in ticker_list:
    df_single = download_stock_data(ticker, period=CONFIG['period'], interval=CONFIG['interval'])
    if df_single is not None:
        try:
            df_features = compute_enhanced_features(df_single)
            all_dfs.append(df_features)
        except Exception as e:
            print(f"Feature engineering failed for {ticker}: {e}")
    else:
        print(f"Failed to download {ticker}")

if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataset created with shape: {df.shape}")
else:
    df = pd.DataFrame()
    print("No data available.")

if not df.empty:
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print("Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")


#Download Function
def download_stock_data(ticker, period="720d", interval="1h", max_retries=5):
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, period=period, interval=interval)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

#Feature Engineering
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())
    df['PROC'] = ((df['Close'] - df['Close'].shift(12)) / df['Close'].shift(12)) * 100
    df['Rolling_Mean_50'] = df['Close'].rolling(50).mean()
    df['Expanding_Mean'] = df['Close'].expanding().mean()
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['MACD_Hist'] = df['MACD_Line'] - df['MACD_Signal']
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = -delta.where(delta < 0, 0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['True_Range'] = df[['High', 'Low', 'Close']].apply(
        lambda x: max(x.iloc[0] - x.iloc[1], abs(x.iloc[0] - x.iloc[2]), abs(x.iloc[1] - x.iloc[2])), axis=1)
    df['ATR'] = df['True_Range'].rolling(14).mean()
    df['+DM'] = np.where((df['High'].diff() > df['Low'].diff()) & (df['High'].diff() > 0), df['High'].diff(), 0)
    df['-DM'] = np.where((df['Low'].diff() > df['High'].diff()) & (df['Low'].diff() > 0), df['Low'].diff(), 0)
    df['+DI'] = 100 * df['+DM'].rolling(14).mean() / df['ATR']
    df['-DI'] = 100 * df['-DM'].rolling(14).mean() / df['ATR']
    df['ADX'] = abs(df['+DI'] - df['-DI']).rolling(14).mean()
    df['Volume_Avg'] = df['Volume'].rolling(20).mean()
    df['Volume_Change'] = df['Volume'].pct_change()
    df['Volume_Change_MA'] = df['Volume_Change'].rolling(10).mean()
    df['Volume_Change_Ratio'] = df['Volume_Change'] / df['Volume_Change'].shift(1)
    df['Relative_Volume'] = df['Volume'] / df['Volume_Avg']
    df['Trailing_Stop'] = np.minimum(df['Close'] * 0.985, df['Close'] - (df['ATR'] * 0.3))
    df['Buy_Signal'] = np.where((df['RSI'] < 60) & (df['EMA_10'] > df['EMA_50']) &
                                ((df['MACD_Line'] > df['MACD_Signal']) | (df['MACD_Line'].diff() > 0)) &
                                (df['Volume'] > (0.4 * df['Volume_Avg'])) & (df['ADX'] > 18), 1, 0)
    df['Sell_Signal'] = np.where(((df['EMA_10'] < df['EMA_50']) & (df['RSI'] > 60)) |
                                 ((df['MACD_Line'] < df['MACD_Signal']) & (df['RSI'] > 65)) |
                                 (df['Close'] < df['Trailing_Stop']) |
                                 ((df['Volume'] > 0.5 * df['Volume_Avg']) & (df['ADX'] > 20)), 1, 0)
    df['Sell_Signal_Debug'] = np.where(((df['MACD_Hist'] < 0.5) | (df['MACD_Line'] < df['MACD_Signal'])) &
                                       (df['RSI'] < 55) & (df['ADX'] > 15) &
                                       ((df['Close'] < df['Trailing_Stop']) | (df['EMA_10'] < df['EMA_50'])) &
                                       (df['Volume'] > 0.5 * df['Volume_Avg']), 1, 0)
    df['Future_Close'] = df['Close'].shift(-10)
    df['Volatility'] = df['Close'].pct_change().rolling(window=20).std()
    df['Return'] = (df['Future_Close'] - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df['Multi_Class_Target'] = df['Target']
    df['Hour'] = pd.to_datetime(df['Datetime']).dt.hour
    df['DayOfWeek'] = pd.to_datetime(df['Datetime']).dt.dayofweek
    df['Session'] = np.where((df['Hour'] >= 9) & (df['Hour'] <= 16), 'Regular',
                             np.where((df['Hour'] < 9), 'Pre-market', 'After-hours'))
    df['MACD_Crossover'] = np.where(df['MACD_Line'] > df['MACD_Signal'], 1, 0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df


Attempt 1: Downloading AAPL...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading TSLA...


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading MSFT...


...

Attempt 1: Downloading SBUX...


[*********************100%***********************]  1 of 1 completed


Attempt 1: Downloading ORCL...


[*********************100%***********************]  1 of 1 completed


Combined dataset created with shape: (262337, 51)
Saved locally to multi_stock_feature_engineered_dataset.csv
Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [13]:
!rm -rf /content/drive

In [16]:
# === Imports ===
import os
import gc
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from google.colab import drive

# === Mount Google Drive ===
drive.mount("/content/drive", force_remount=True)

# === Configurations ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/xgb_walkforward_results"
FINAL_MODEL_DIR = "/content/drive/MyDrive/xgb_walkforward_results/models"
os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/data", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/models", exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Load Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

features = ['SMA_20', 'STD_20', 'Upper_Band', 'Lower_Band', 'Stoch']
target = "Target"
label_map = {-1: 0, 0: 1, 1: 2}
results = []

# === Rolling Window Generator ===
def generate_date_windows(start_date, end_date, train_days=365, test_days=60, step_days=60):
    windows = []
    current = pd.to_datetime(start_date)
    while current + timedelta(days=train_days + test_days) <= pd.to_datetime(end_date):
        train_start = current
        train_end = train_start + timedelta(days=train_days)
        test_start = train_end
        test_end = test_start + timedelta(days=test_days)
        windows.append((train_start, train_end, test_start, test_end))
        current += timedelta(days=step_days)
    return windows

# === Walkforward XGB ===
def walkforward_xgb(df_ticker, ticker):
    df_ticker = df_ticker.dropna(subset=features + [target]).copy()
    df_ticker['Target_Mapped'] = df_ticker[target].map(label_map)
    windows = generate_date_windows("2020-01-01", "2024-01-01")

    for (train_start, train_end, test_start, test_end) in windows:
        train_df = df_ticker[(df_ticker['Datetime'] >= train_start) & (df_ticker['Datetime'] < train_end)]
        test_df = df_ticker[(df_ticker['Datetime'] >= test_start) & (df_ticker['Datetime'] < test_end)]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        X_train, y_train = train_df[features], train_df['Target_Mapped']
        X_test, y_test = test_df[features], test_df['Target_Mapped']

        model = XGBClassifier(n_estimators=50, learning_rate=0.1, tree_method='hist', random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)

        test_df = test_df.copy()
        test_df['Signal'] = preds - 1

        capital = 100000
        shares = 0
        portfolio = []

        for _, row in test_df.iterrows():
            price = row['Close']
            signal = row['Signal']
            if signal == 1 and capital >= price and shares == 0:
                shares = (capital * 0.05) // price
                capital -= shares * price
            elif signal == -1 and shares > 0:
                capital += shares * price
                shares = 0
            portfolio.append(capital + shares * price)

        if not portfolio:
            continue

        final_value = portfolio[-1]
        return_pct = (final_value - 100000) / 100000 * 100
        returns = pd.Series(portfolio).pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

        results.append({
            "Ticker": ticker,
            "Train Period": f"{train_start.date()} to {train_end.date()}",
            "Test Period": f"{test_start.date()} to {test_end.date()}",
            "Model": "XGBoost",
            "Accuracy": round(acc, 4),
            "Sharpe": round(sharpe, 3),
            "Drawdown": round(drawdown, 2),
            "Return": round(return_pct, 2),
            "Final_Portfolio": round(final_value, 2)
        })

        filename_prefix = f"{ticker}_{train_start.date()}_{test_start.date()}"

        try:
            joblib.dump(model, os.path.join(FINAL_MODEL_DIR, f"xgb_{filename_prefix}.pkl"))
            with open(os.path.join(FINAL_MODEL_DIR, f"xgb_{filename_prefix}_features.json"), "w") as f:
                json.dump(features, f)
            print(f"✅ Model saved: xgb_{filename_prefix}.pkl")
        except Exception as e:
            print(f"❌ Failed to save model for {ticker}: {e}")

        test_df.to_csv(os.path.join(RESULTS_DIR, "data", f"{ticker}_{filename_prefix}_result.csv"), index=False)

        plt.figure(figsize=(12, 6))
        plt.plot(test_df['Close'].values, label='Close Price')
        plt.plot(test_df['Signal'].cumsum(), label='Cumulative Signal')
        plt.title(f"{ticker} - XGB Strategy Signals")
        plt.xlabel("Time Steps")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, "plots", f"{ticker}_{filename_prefix}_xgb_signals.png"))
        plt.close()

# === Run All Tickers ===
tickers = df['Symbol'].unique()
for ticker in tickers:
    print(f"\n Processing {ticker}")
    df_ticker = df[df['Symbol'] == ticker].copy()
    if len(df_ticker) < 1000:
        print(f"⚠️ Skipping {ticker}, not enough data.")
        continue
    walkforward_xgb(df_ticker, ticker)
    gc.collect()

# === Save Summary ===
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(RESULTS_DIR, "xgb_walkforward_metrics.csv"), index=False)

summary_df['score'] = (
    summary_df['Sharpe'] * 0.4 +
    summary_df['Return'] * 0.3 +
    summary_df['Final_Portfolio'] * 0.3
)
summary_df.to_csv(os.path.join(RESULTS_DIR, "xgb_model_selector_metrics.csv"), index=False)

best_models = summary_df.sort_values(['Ticker', 'score'], ascending=[True, False])\
                        .groupby('Ticker').first().reset_index()
best_models.to_excel(os.path.join(RESULTS_DIR, "xgb_best_models_by_score.xlsx"), index=False)


Mounted at /content/drive

 Processing AAPL
✅ Model saved: xgb_AAPL_2021-10-22_2022-10-22.pkl
✅ Model saved: xgb_AAPL_2021-12-21_2022-12-21.pkl
✅ Model saved: xgb_AAPL_2022-02-19_2023-02-19.pkl
✅ Model saved: xgb_AAPL_2022-04-20_2023-04-20.pkl
✅ Model saved: xgb_AAPL_2022-06-19_2023-06-19.pkl
✅ Model saved: xgb_AAPL_2022-08-18_2023-08-18.pkl
✅ Model saved: xgb_AAPL_2022-10-17_2023-10-17.pkl

 Processing TSLA
✅ Model saved: xgb_TSLA_2021-10-22_2022-10-22.pkl
✅ Model saved: xgb_TSLA_2021-12-21_2022-12-21.pkl
✅ Model saved: xgb_TSLA_2022-02-19_2023-02-19.pkl
✅ Model saved: xgb_TSLA_2022-04-20_2023-04-20.pkl
✅ Model saved: xgb_TSLA_2022-06-19_2023-06-19.pkl
✅ Model saved: xgb_TSLA_2022-08-18_2023-08-18.pkl
✅ Model saved: xgb_TSLA_2022-10-17_2023-10-17.pkl

 Processing MSFT
✅ Model saved: xgb_MSFT_2021-10-22_2022-10-22.pkl
✅ Model saved: xgb_MSFT_2021-12-21_2022-12-21.pkl
✅ Model saved: xgb_MSFT_2022-02-19_2023-02-19.pkl
✅ Model saved: xgb_MSFT_2022-04-20_2023-04-20.pkl
✅ Model saved: xgb_M

In [32]:
# === Imports ===
import os
import gc
import json
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# === Mount Google Drive ===
drive.mount("/content/drive", force_remount=True)

# === Config ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/rf_walkforward_results"
FINAL_MODEL_DIR = "/content/drive/MyDrive/Results_May_2025/final_combined_results/models"
os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/data", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/models", exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Load Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

features = ['SMA_20', 'STD_20', 'Upper_Band', 'Lower_Band', 'Stoch']
target = "Target"
label_map = {-1: 0, 0: 1, 1: 2}
results = []

# === Rolling Window Generator ===
def generate_date_windows(start_date, end_date, train_days=365, test_days=60, step_days=60):
    windows = []
    current = pd.to_datetime(start_date)
    while current + timedelta(days=train_days + test_days) <= pd.to_datetime(end_date):
        train_start = current
        train_end = train_start + timedelta(days=train_days)
        test_start = train_end
        test_end = test_start + timedelta(days=test_days)
        windows.append((train_start, train_end, test_start, test_end))
        current += timedelta(days=step_days)
    return windows

# === Walkforward Training ===
def walkforward_rf(df_ticker, ticker):
    df_ticker = df_ticker.dropna(subset=features + [target]).copy()
    df_ticker['Target_Mapped'] = df_ticker[target].map(label_map)
    windows = generate_date_windows("2020-01-01", "2024-01-01")

    for (train_start, train_end, test_start, test_end) in windows:
        train_df = df_ticker[(df_ticker['Datetime'] >= train_start) & (df_ticker['Datetime'] < train_end)]
        test_df = df_ticker[(df_ticker['Datetime'] >= test_start) & (df_ticker['Datetime'] < test_end)].copy()

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_df[features])
        X_test = scaler.transform(test_df[features])

        y_train = train_df['Target_Mapped']
        y_test = test_df['Target_Mapped']

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        test_df.loc[:, 'Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
        test_df.loc[:, 'Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})

        acc = accuracy_score(y_test, preds)
        print(f"[RF] {ticker} | Train={len(train_df)}, Test={len(test_df)}, Acc={acc:.3f}")
        print("Signal counts:", test_df['Signal'].value_counts().to_dict())

        capital = 100000
        shares = 0
        portfolio = []

        for _, row in test_df.iterrows():
            price = row['Close']
            signal = row['Signal']

            if signal == 1 and capital >= price and shares == 0:
                amount = capital * 0.05
                shares = int(amount // price)
                if shares > 0:
                    capital -= shares * price * (1 + 0.001)
                    print(f"BUY @ {price:.2f} | Shares={shares} | Capital={capital:.2f}")
            elif signal == -1 and shares > 0:
                capital += shares * price * (1 - 0.001)
                print(f"SELL @ {price:.2f} | Shares={shares} | Capital={capital:.2f}")
                shares = 0

            portfolio.append(capital + shares * price)

        if not portfolio or len(portfolio) < 2:
            print(f"[⚠️] No trades or insufficient portfolio data for {ticker} window {test_start.date()}")
            continue

        final_value = portfolio[-1]
        return_pct = (final_value - 100000) / 100000 * 100
        hold_value = (100000 / test_df['Close'].iloc[0]) * test_df['Close'].iloc[-1]
        returns = pd.Series(portfolio).pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

        if sharpe == 0.0 and return_pct == 0.0 and acc == 0.0:
            print(f"[⚠️] Metrics are zero for {ticker} — skipping result")
            continue
        if final_value > 300000:
            print(f"[⚠️] Unrealistically high portfolio for {ticker}: ${final_value:.2f} — skipping")
            continue

        results.append({
            "Ticker": ticker,
            "Train Period": f"{train_start.date()} to {train_end.date()}",
            "Test Period": f"{test_start.date()} to {test_end.date()}",
            "Model": "Random Forest",
            "Accuracy": round(acc, 4),
            "Sharpe": round(sharpe, 3),
            "Drawdown": round(drawdown, 2),
            "Return": round(return_pct, 2),
            "Final_Portfolio": round(final_value, 2)
        })

        filename_prefix = f"{ticker}_{train_start.date()}_{test_start.date()}"
        joblib.dump(model, os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}.pkl"))
        joblib.dump(scaler, os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}_scaler.pkl"))
        with open(os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}_features.json"), "w") as f:
            json.dump(features, f)
        test_df.to_csv(os.path.join(RESULTS_DIR, "data", f"{ticker}_{filename_prefix}_result.csv"), index=False)

        plt.figure(figsize=(12, 6))
        plt.plot(test_df['Close'].values, label='Close Price')
        plt.plot(test_df['Signal'].cumsum(), label='Cumulative Signal')
        plt.title(f"{ticker} - RF Strategy Signals")
        plt.xlabel("Time Steps")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, "plots", f"{ticker}_{filename_prefix}_rf_signals.png"))
        plt.close()

        print(f"[RF] Final Portfolio: ${final_value:.2f} | Hold: ${hold_value:.2f} | Sharpe: {sharpe:.2f}")

# === Run All Tickers ===
tickers = df['Symbol'].unique()
for ticker in tickers:
    print(f"\n🚀 Processing {ticker}")
    df_ticker = df[df['Symbol'] == ticker].copy()
    if len(df_ticker) < 1000:
        print(f"⚠️ Skipping {ticker}, not enough data.")
        continue
    walkforward_rf(df_ticker, ticker)
    gc.collect()

# === Save Summary ===
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(RESULTS_DIR, "rf_walkforward_metrics.csv"), index=False)

summary_df.fillna(0, inplace=True)
summary_df['score'] = (
    summary_df['Sharpe'] * 0.4 +
    summary_df['Return'] * 0.3 +
    summary_df['Final_Portfolio'] * 0.3
)
summary_df.to_csv(os.path.join(RESULTS_DIR, "rf_model_selector_metrics.csv"), index=False)

best_models = summary_df.sort_values(['Ticker', 'score'], ascending=[True, False])\
                        .groupby('Ticker').first().reset_index()
best_models.to_excel(os.path.join(RESULTS_DIR, "rf_best_models_by_score.xlsx"), index=False)


Mounted at /content/drive

🚀 Processing AAPL
[RF] AAPL | Train=420, Test=283, Acc=0.452
Signal counts: {1: 110, 0: 98, -1: 75}
BUY @ 148.25 | Shares=33 | Capital=95102.86
SELL @ 150.59 | Shares=33 | Capital=100067.36
BUY @ 150.57 | Shares=33 | Capital=95093.58
SELL @ 148.99 | Shares=33 | Capital=100005.33
BUY @ 148.86 | Shares=33 | Capital=95088.04
SELL @ 138.88 | Shares=33 | Capital=99666.50
BUY @ 149.71 | Shares=33 | Capital=94720.96
SELL @ 148.34 | Shares=33 | Capital=99611.29
BUY @ 146.23 | Shares=34 | Capital=94634.49
SELL @ 136.96 | Shares=34 | Capital=99286.65
[RF] Final Portfolio: $99286.65 | Hold: $90220.15 | Sharpe: -1.03
[RF] AAPL | Train=703, Test=280, Acc=0.350
Signal counts: {1: 159, -1: 97, 0: 24}
BUY @ 134.89 | Shares=37 | Capital=95004.08
SELL @ 135.50 | Shares=37 | Capital=100012.57
BUY @ 130.17 | Shares=38 | Capital=95061.34
SELL @ 126.19 | Shares=38 | Capital=99851.77
BUY @ 129.39 | Shares=38 | Capital=94930.03
SELL @ 124.66 | Shares=38 | Capital=99662.37
BUY @ 124.

In [30]:
# === Imports ===
import os
import gc
import json
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# === Mount Google Drive ===
drive.mount("/content/drive", force_remount=True)

# === Config ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/rf_walkforward_results"
FINAL_MODEL_DIR = "/content/drive/MyDrive/Results_May_2025/final_combined_results/models"
os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/data", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/models", exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Load Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

features = ['SMA_20', 'STD_20', 'Upper_Band', 'Lower_Band', 'Stoch']
target = "Target"
label_map = {-1: 0, 0: 1, 1: 2}
results = []

# === Rolling Window Generator ===
def generate_date_windows(start_date, end_date, train_days=365, test_days=60, step_days=60):
    windows = []
    current = pd.to_datetime(start_date)
    while current + timedelta(days=train_days + test_days) <= pd.to_datetime(end_date):
        train_start = current
        train_end = train_start + timedelta(days=train_days)
        test_start = train_end
        test_end = test_start + timedelta(days=test_days)
        windows.append((train_start, train_end, test_start, test_end))
        current += timedelta(days=step_days)
    return windows

# === Walkforward Training ===
def walkforward_rf(df_ticker, ticker):
    df_ticker = df_ticker.dropna(subset=features + [target]).copy()
    df_ticker['Target_Mapped'] = df_ticker[target].map(label_map)
    windows = generate_date_windows("2020-01-01", "2024-01-01")

    for (train_start, train_end, test_start, test_end) in windows:
        train_df = df_ticker[(df_ticker['Datetime'] >= train_start) & (df_ticker['Datetime'] < train_end)]
        test_df = df_ticker[(df_ticker['Datetime'] >= test_start) & (df_ticker['Datetime'] < test_end)]

        if len(train_df) < 200 or len(test_df) < 50:
            continue

        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_df[features])
        X_test = scaler.transform(test_df[features])

        y_train = train_df['Target_Mapped']
        y_test = test_df['Target_Mapped']

        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        preds = model.predict(X_test)
        test_df['Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
        test_df['Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})

        acc = accuracy_score(y_test, preds)
        print(f"[RF] {ticker} | Train={len(train_df)}, Test={len(test_df)}, Acc={acc:.3f}")
        print("Signal counts:", test_df['Signal'].value_counts().to_dict())

        capital = 100000
        shares = 0
        portfolio = []

        for _, row in test_df.iterrows():
            price = row['Close']
            signal = row['Signal']

            if signal == 1 and capital >= price and shares == 0:
                amount = capital * 0.05
                shares = int(amount // price)
                if shares > 0:
                    capital -= shares * price * (1 + 0.001)
                    print(f"BUY @ {price:.2f} | Shares={shares} | Capital={capital:.2f}")
            elif signal == -1 and shares > 0:
                capital += shares * price * (1 - 0.001)
                print(f"SELL @ {price:.2f} | Shares={shares} | Capital={capital:.2f}")
                shares = 0

            portfolio.append(capital + shares * price)

        if not portfolio or len(portfolio) < 2:
            print(f"[⚠️] No trades or insufficient portfolio data for {ticker} window {test_start.date()}")
            continue

        final_value = portfolio[-1]
        return_pct = (final_value - 100000) / 100000 * 100
        hold_value = (100000 / test_df['Close'].iloc[0]) * test_df['Close'].iloc[-1]
        returns = pd.Series(portfolio).pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

        # Guard against invalid or unrealistic values
        if sharpe == 0.0 and return_pct == 0.0 and acc == 0.0:
            print(f"[⚠️] Metrics are zero for {ticker} — skipping result")
            continue
        if final_value > 300000:
            print(f"[⚠️] Unrealistically high portfolio for {ticker}: ${final_value:.2f} — skipping")
            continue

        results.append({
            "Ticker": ticker,
            "Train Period": f"{train_start.date()} to {train_end.date()}",
            "Test Period": f"{test_start.date()} to {test_end.date()}",
            "Model": "Random Forest",
            "Accuracy": round(acc, 4),
            "Sharpe": round(sharpe, 3),
            "Drawdown": round(drawdown, 2),
            "Return": round(return_pct, 2),
            "Final_Portfolio": round(final_value, 2)
        })

        filename_prefix = f"{ticker}_{train_start.date()}_{test_start.date()}"
        joblib.dump(model, os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}.pkl"))
        joblib.dump(scaler, os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}_scaler.pkl"))
        with open(os.path.join(FINAL_MODEL_DIR, f"rf_{filename_prefix}_features.json"), "w") as f:
            json.dump(features, f)
        test_df.to_csv(os.path.join(RESULTS_DIR, "data", f"{ticker}_{filename_prefix}_result.csv"), index=False)

        plt.figure(figsize=(12, 6))
        plt.plot(test_df['Close'].values, label='Close Price')
        plt.plot(test_df['Signal'].cumsum(), label='Cumulative Signal')
        plt.title(f"{ticker} - RF Strategy Signals")
        plt.xlabel("Time Steps")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, "plots", f"{ticker}_{filename_prefix}_rf_signals.png"))
        plt.close()

        print(f"[RF] Final Portfolio: ${final_value:.2f} | Hold: ${hold_value:.2f} | Sharpe: {sharpe:.2f}")

# === Run All Tickers ===
tickers = df['Symbol'].unique()
for ticker in tickers:
    print(f"\n🚀 Processing {ticker}")
    df_ticker = df[df['Symbol'] == ticker].copy()
    if len(df_ticker) < 1000:
        print(f"⚠️ Skipping {ticker}, not enough data.")
        continue
    walkforward_rf(df_ticker, ticker)
    gc.collect()

# === Save Summary ===
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(RESULTS_DIR, "rf_walkforward_metrics.csv"), index=False)

summary_df.fillna(0, inplace=True)
summary_df['score'] = (
    summary_df['Sharpe'] * 0.4 +
    summary_df['Return'] * 0.3 +
    summary_df['Final_Portfolio'] * 0.3
)
summary_df.to_csv(os.path.join(RESULTS_DIR, "rf_model_selector_metrics.csv"), index=False)

best_models = summary_df.sort_values(['Ticker', 'score'], ascending=[True, False])\
                        .groupby('Ticker').first().reset_index()
best_models.to_excel(os.path.join(RESULTS_DIR, "rf_best_models_by_score.xlsx"), index=False)


Mounted at /content/drive

🚀 Processing AAPL


<ipython-input-30-a5d2a5df5ee2>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
<ipython-input-30-a5d2a5df5ee2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})


[RF] AAPL | Train=420, Test=283, Acc=0.452
Signal counts: {1: 110, 0: 98, -1: 75}
BUY @ 148.25 | Shares=33 | Capital=95102.86
SELL @ 150.59 | Shares=33 | Capital=100067.36
BUY @ 150.57 | Shares=33 | Capital=95093.58
SELL @ 148.99 | Shares=33 | Capital=100005.33
BUY @ 148.86 | Shares=33 | Capital=95088.04
SELL @ 138.88 | Shares=33 | Capital=99666.50
BUY @ 149.71 | Shares=33 | Capital=94720.96
SELL @ 148.34 | Shares=33 | Capital=99611.29
BUY @ 146.23 | Shares=34 | Capital=94634.49
SELL @ 136.96 | Shares=34 | Capital=99286.65
[RF] Final Portfolio: $99286.65 | Hold: $90220.15 | Sharpe: -1.03


<ipython-input-30-a5d2a5df5ee2>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
<ipython-input-30-a5d2a5df5ee2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})


[RF] AAPL | Train=703, Test=280, Acc=0.350
Signal counts: {1: 159, -1: 97, 0: 24}
BUY @ 134.89 | Shares=37 | Capital=95004.08
SELL @ 135.50 | Shares=37 | Capital=100012.57
BUY @ 130.17 | Shares=38 | Capital=95061.34
SELL @ 126.19 | Shares=38 | Capital=99851.77
BUY @ 129.39 | Shares=38 | Capital=94930.03
SELL @ 124.66 | Shares=38 | Capital=99662.37
BUY @ 124.65 | Shares=39 | Capital=94796.17
SELL @ 135.58 | Shares=39 | Capital=100078.61
BUY @ 135.88 | Shares=36 | Capital=95181.86
SELL @ 135.30 | Shares=36 | Capital=100047.79
BUY @ 135.54 | Shares=36 | Capital=95163.47
SELL @ 140.47 | Shares=36 | Capital=100215.25
BUY @ 142.42 | Shares=35 | Capital=95225.55
SELL @ 143.52 | Shares=35 | Capital=100243.91
BUY @ 141.96 | Shares=35 | Capital=95270.34
SELL @ 142.39 | Shares=35 | Capital=100249.00
BUY @ 145.53 | Shares=34 | Capital=95296.03
SELL @ 151.68 | Shares=34 | Capital=100448.00
BUY @ 153.84 | Shares=32 | Capital=95520.19
SELL @ 152.94 | Shares=32 | Capital=100409.38
BUY @ 154.65 | Share

...

<ipython-input-30-a5d2a5df5ee2>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
<ipython-input-30-a5d2a5df5ee2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})


[RF] MSFT | Train=1270, Test=287, Acc=0.714
Signal counts: {1: 266, 0: 20, -1: 1}
BUY @ 287.16 | Shares=17 | Capital=95113.40
SELL @ 281.76 | Shares=17 | Capital=99898.53
BUY @ 280.04 | Shares=17 | Capital=95133.09
[RF] Final Portfolio: $100956.61 | Hold: $119292.38 | Sharpe: 1.70


<ipython-input-30-a5d2a5df5ee2>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Predicted'] = pd.Series(preds, index=test_df.index).astype(int)
<ipython-input-30-a5d2a5df5ee2>:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Signal'] = test_df['Predicted'].map({0: 0, 1: 1, 2: -1})


[RF] MSFT | Train=1557, Test=290, Acc=0.531
Signal counts: {0: 162, 1: 121, -1: 7}
BUY @ 330.22 | Shares=15 | Capital=95041.75
SELL @ 336.80 | Shares=15 | Capital=100088.69
BUY @ 339.89 | Shares=14 | Capital=95325.48
SELL @ 332.33 | Shares=14 | Capital=99973.44
BUY @ 329.79 | Shares=15 | Capital=95021.65
SELL @ 316.88 | Shares=15 | Capital=99770.09
[RF] Final Portfolio: $99770.09 | Hold: $94202.98 | Sharpe: -0.39


KeyboardInterrupt: 